# DSM on SUPPORT Dataset

The SUPPORT dataset comes from the Vanderbilt University study
to estimate survival for seriously ill hospitalized adults.
(Refer to http://biostat.mc.vanderbilt.edu/wiki/Main/SupportDesc.
for the original datasource.)

In this notebook, we will apply Deep Survival Machines for survival prediction on the SUPPORT data.

### Load the SUPPORT Dataset

The package includes helper functions to load the dataset.

X represents an np.array of features (covariates),
T is the event/censoring times and,
E is the censoring indicator.

In [21]:
import os


a = os.getcwd()
%cd c:/Users/tiant/OneDrive/桌面/IntroToDL/project/surv-rcts/



c:\Users\tiant\OneDrive\桌面\IntroToDL\project\surv-rcts


In [22]:
from auton_survival.models.dsm import datasets
x, t, e = datasets.load_dataset('SUPPORT')

### Compute horizons at which we evaluate the performance of DSM

Survival predictions are issued at certain time horizons. Here we will evaluate the performance
of DSM to issue predictions at the 25th, 50th and 75th event time quantile as is standard practice in Survival Analysis.

In [59]:
import numpy as np
horizons = [0.25, 0.5, 0.75]
times = np.quantile(t[e==1], horizons).tolist()
# times =  [365.0,730.0,1825.0]

### Splitting the data into train, test and validation sets

We will train DSM on 70% of the Data, use a Validation set of 10% for Model Selection and report performance on the remaining 20% held out test set.

In [34]:
n = len(x)

tr_size = int(n*0.70)
vl_size = int(n*0.10)
te_size = int(n*0.20)

x_train, x_test, x_val = x[:tr_size], x[-te_size:], x[tr_size:tr_size+vl_size]
t_train, t_test, t_val = t[:tr_size], t[-te_size:], t[tr_size:tr_size+vl_size]
e_train, e_test, e_val = e[:tr_size], e[-te_size:], e[tr_size:tr_size+vl_size]

### Setting the parameter grid

Lets set up the parameter grid to tune hyper-parameters. We will tune the number of underlying survival distributions, 
($K$), the distribution choices (Log-Normal or Weibull), the learning rate for the Adam optimizer between $1\times10^{-3}$ and $1\times10^{-4}$ and the number of hidden layers between $0, 1$ and $2$.

In [35]:
from sklearn.model_selection import ParameterGrid

In [36]:
param_grid = {'k' : [3, 4, 6],
              'distribution' : ['LogNormal', 'Weibull'],
              'learning_rate' : [ 1e-4, 1e-3],
              'layers' : [ [], [100], [100, 100] ]
             }
params = ParameterGrid(param_grid)

In [46]:
et_train = np.array([(e_train[i], t_train[i]) for i in range(len(e_train))],
                 dtype = [('e', bool), ('t', float)])
et_test = np.array([(e_test[i], t_test[i]) for i in range(len(e_test))],
                 dtype = [('e', bool), ('t', float)])
et_val = np.array([(e_val[i], t_val[i]) for i in range(len(e_val))],
                 dtype = [('e', bool), ('t', float)])

### Model Training and Selection

In [47]:
from models.dsm import DeepSurvivalMachines

In [51]:

from sksurv.metrics import concordance_index_ipcw, brier_score, cumulative_dynamic_auc

In [57]:
import sksurv

In [60]:
models = []
# temp_pred_survival = []
brier_score_li = []
for param in params:
    model = DeepSurvivalMachines(k = param['k'],
                                 distribution = param['distribution'],
                                 layers = param['layers'])
    # The fit method is called to train the model
    model.fit(x_train, t_train, e_train, iters = 100, learning_rate = param['learning_rate'])
    temp_pred_survival=model.predict_survival(x_val, times)
    brier_score_li.append(sksurv.metrics.brier_score(et_train, et_val,temp_pred_survival,times)[1])
    models.append(model)
min_id = np.argmin(np.mean(brier_score_li, axis = 1))
best_model = models[min_id]
model = best_model

  9%|▉         | 9/100 [00:02<00:25,  3.53it/s]


### Inference

In [61]:
out_risk = model.predict_risk(x_test, times)
out_survival = model.predict_survival(x_test, times)

### Evaluation

We evaluate the performance of DSM in its discriminative ability (Time Dependent Concordance Index and Cumulative Dynamic AUC) as well as Brier Score.

In [40]:
from sksurv.metrics import concordance_index_ipcw, brier_score, cumulative_dynamic_auc

In [64]:
cis = []
brs = []
check = []
et_train = np.array([(e_train[i], t_train[i]) for i in range(len(e_train))],
                 dtype = [('e', bool), ('t', float)])
et_test = np.array([(e_test[i], t_test[i]) for i in range(len(e_test))],
                 dtype = [('e', bool), ('t', float)])
et_val = np.array([(e_val[i], t_val[i]) for i in range(len(e_val))],
                 dtype = [('e', bool), ('t', float)])

for i, _ in enumerate(times):
    cis.append(concordance_index_ipcw(et_train, et_test, out_risk[:, i], times[i])[0])
    # check.append(brier_score(et_train, et_test, out_survival, times))
brs.append(sksurv.metrics.brier_score(et_train, et_test, out_survival, times)[1])
roc_auc = []
for i, _ in enumerate(times):
    roc_auc.append(cumulative_dynamic_auc(et_train, et_test, out_risk[:, i], times[i])[0])
for horizon in enumerate(times):
    print(f"For {horizon[1]} quantile,")
    print("TD Concordance Index:", cis[horizon[0]])
    print("Brier Score:", brs[0][horizon[0]])
    print("ROC AUC ", roc_auc[horizon[0]][0], "\n")
    print('check',check)

For 14.0 quantile,
TD Concordance Index: 0.769235123736921
Brier Score: 0.10683213723233631
ROC AUC  0.7799235172282094 

check []
For 58.0 quantile,
TD Concordance Index: 0.7022526475634527
Brier Score: 0.1788751530648107
ROC AUC  0.7248558672572594 

check []
For 252.0 quantile,
TD Concordance Index: 0.6683970644775017
Brier Score: 0.21518664042971875
ROC AUC  0.717063064154601 

check []
